# RRAP-IS Authentication Demo Notebook

> A tutorial of RRAP authentication using Jupyter notebooks.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter, authentication]

## About

This notebook is a demonstration of authenticating with the RRAP-IS system.  

## Run all imports

Keep all your imports at the top of a notebook.  It allows for easier management.

In [ ]:
import requests
import os
import sys
import json
from IPython.core.display import display, HTML
from IPython.display import IFrame
sys.path.append("python_packages/rrap_packages")
    
from python_packages.rrap_packages.token_manager import DeviceFlowManager

## Define global variables

Similar to import we like to define notebook variable at the top and reuse them throughout the notebook

In [ ]:
data_api = "https://data-api.testing.rrap-is.com"
registry_api = "https://registry-api.testing.rrap-is.com"
prov_api = "https://prov-api.testing.rrap-is.com"
auth_server = "https://auth.dev.rrap-is.com/auth/realms/rrap"
# garbage = "https://frogs.are.green"
base_urls = {'data_api': data_api, 'registry_api': registry_api, 'prov_api': prov_api, 'auth_server': auth_server}#, 'garbage': garbage}
display(f'Checking base urls')

for key, url in base_urls.items():
    try:
        print(f'Testing - {url}', end="")
        r = requests.get(url)
        r.raise_for_status()
        print(f' - Passed')
    except requests.exceptions.HTTPError as err:
        print(f' - Fail')
        raise SystemExit(err)
    except requests.exceptions.RequestException as e:
        # catastrophic error. bail.
        print(f' - Fail')
        raise SystemExit(e)

## Authentication steps

### Setup tokens using device authorisation flow against keycloak server

This could result in a browser window being opened if you don't have valid tokens cached in local storage.

In [ ]:
# this caches the tokens
local_token_storage = ".tokens.json"

token_manager = DeviceFlowManager(
    stage="TEST",
    keycloak_endpoint=auth_server,
    local_storage_location=local_token_storage
)

### Authenticated call

In [ ]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/items/list-all-datasets"
endpoint = data_api + postfix

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

## Test your auth against a endpoint 
Endpoint documentation can be found by appending either `/docs` or `/redoc` on the end a base URL.

For example:
- https://registry-api.testing.rrap-is.com/redoc
or
- https://registry-api.testing.rrap-is.com/docs

Then select from the menu an endpoint function call e.g. `/check-access/check-general-access`

Then append the function call onto the base url e.g. `https://registry-api.testing.rrap-is.com/check-access/check-general-access`

### Endpoint documentation redoc

In [ ]:
IFrame("https://registry-api.testing.rrap-is.com/redoc", width=1024, height=600)

## Testing General Access Endpoint

In [ ]:
#test general access endpoint
auth = token_manager.get_auth
base_url = "https://registry-api.testing.rrap-is.com"
url_query_string = "/check-access/check-general-access"
endpoint = base_url + url_query_string

response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

In [ ]:
#test read access endpoint
auth = token_manager.get_auth
base_url = "https://registry-api.testing.rrap-is.com"
url_query_string = "/check-access/check-read-access"
endpoint = base_url + url_query_string

response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

In [ ]:
#test write access endpoint
auth = token_manager.get_auth
base_url = "https://registry-api.testing.rrap-is.com"
url_query_string = "/check-access/check-write-access"
endpoint = base_url + url_query_string

response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

In [ ]:
#test admin access endpoint
auth = token_manager.get_auth
base_url = "https://registry-api.testing.rrap-is.com"
url_query_string = "/check-access/check-admin-access"
endpoint = base_url + url_query_string

response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))